In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import sklearn

#X_train, X_test, y_train, y_test = train_test_split(title, label, test_size=0.1, random_state=42)

C:\Users\28529\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def getOnePredict(source):
    result = []
    for i in source:
        result_item = np.zeros(i.size)
        result_item[i.argmax()] = 1
        result.append(result_item)
    return np.array(result)

In [3]:
def onehot2index(source):
    result = np.zeros(source.shape[0])
    for i in range(source.shape[0]):
        result[i] = source[i].argmax()
    return result

In [15]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
#get word level data
def getData():
    label = []
    data = []
    with open('train_a_c.txt') as f:
        isFirst = True
        count = 1
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue            

            if not line:
                break

            l = line.split('\t')

            label.append(int(l[5]))
            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
            
            
            
            data.append(new_result)
            if count%10000 == 0:
                print(count)
            count+=1
    
    from sklearn.preprocessing import OneHotEncoder
    train_label = np.array(label)
    ohe = OneHotEncoder()
    ohe.fit([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]])
    train_label = ohe.transform(train_label.reshape(140561,1)).toarray()
    
    return (np.array(data),train_label)


# 注意187的超参是由先得到数据之后再获得的

In [8]:
(data,label) = getData()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [10]:
label.shape

(140561, 10)

In [12]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.05, random_state=42)

In [13]:
import keras

In [26]:
model = keras.models.load_model('E:history1_898903234464271.h5', custom_objects={"f1": f1})

predictPro = model.predict(x_test)
print(onehot2index(predictPro))
print(onehot2index(y_test))
sklearn.metrics.f1_score(onehot2index(y_test),onehot2index(predictPro),average='macro')

[4. 6. 9. ... 6. 6. 6.]
[4. 6. 9. ... 6. 4. 6.]


0.845521691099318

In [17]:
from keras.layers import BatchNormalization

main_input = Input(shape=(187,), dtype='float64')
embedder = Embedding(200 + 1, 256, input_length = 187)
embed = embedder(main_input)

# cnn1模块，kernel_size = 3
conv1_1 = Convolution1D(256, 3, padding='same')(embed)
bn1_1 = BatchNormalization()(conv1_1)
relu1_1 = Activation('relu')(bn1_1)
conv1_2 = Convolution1D(128, 3, padding='same')(relu1_1)
bn1_2 = BatchNormalization()(conv1_2)
relu1_2 = Activation('relu')(bn1_2)
cnn1 = MaxPool1D(pool_size=4)(relu1_2)

# cnn2模块，kernel_size = 4
conv2_1 = Convolution1D(256, 4, padding='same')(embed)
bn2_1 = BatchNormalization()(conv2_1)
relu2_1 = Activation('relu')(bn2_1)
conv2_2 = Convolution1D(128, 4, padding='same')(relu2_1)
bn2_2 = BatchNormalization()(conv2_2)
relu2_2 = Activation('relu')(bn2_2)
cnn2 = MaxPool1D(pool_size=4)(relu2_2)

# cnn3模块，kernel_size = 5
conv3_1 = Convolution1D(256, 5, padding='same')(embed)
bn3_1 = BatchNormalization()(conv3_1)
relu3_1 = Activation('relu')(bn3_1)
conv3_2 = Convolution1D(128, 5, padding='same')(relu3_1)
bn3_2 = BatchNormalization()(conv3_2)
relu3_2 = Activation('relu')(bn3_2)
cnn3 = MaxPool1D(pool_size=4)(relu3_2)

# 拼接三个模块
cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
flat = Flatten()(cnn)

# 此处concantenate上强监督

drop = Dropout(0.5)(flat)
fc = Dense(512)(drop)
bn = BatchNormalization()(fc)
main_output = Dense(10, activation='softmax')(bn)

model = Model(inputs = main_input, outputs = main_output)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[f1])

history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_data=(x_test, y_test))

Train on 133532 samples, validate on 7029 samples
Epoch 1/5
133532/133532 [==============================] - 260s 2ms/step - loss: 0.9282 - f1: 0.6794 - val_loss: 0.5624 - val_f1: 0.8156
Epoch 2/5
133532/133532 [==============================] - 255s 2ms/step - loss: 0.5230 - f1: 0.8293 - val_loss: 0.4693 - val_f1: 0.8481
Epoch 3/5
133532/133532 [==============================] - 259s 2ms/step - loss: 0.4681 - f1: 0.8476 - val_loss: 0.4374 - val_f1: 0.8571
Epoch 4/5
133532/133532 [==============================] - 261s 2ms/step - loss: 0.4612 - f1: 0.8483 - val_loss: 0.6914 - val_f1: 0.7876
Epoch 5/5
133532/133532 [==============================] - 261s 2ms/step - loss: 0.4373 - f1: 0.8565 - val_loss: 0.4531 - val_f1: 0.8588


{}

In [20]:
model.save('E:history1.h5')

In [21]:
while True:
    history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=2)
    name = 'E:history1_' + str(history.history['f1'][-1])[2:]+'.h5'
    model.save(name)

Epoch 1/2
133532/133532 [==============================] - 259s 2ms/step - loss: 0.4104 - f1: 0.8649
Epoch 2/2
133532/133532 [==============================] - 255s 2ms/step - loss: 0.3960 - f1: 0.8699
Epoch 1/2
133532/133532 [==============================] - 255s 2ms/step - loss: 0.3809 - f1: 0.8739
Epoch 2/2
133532/133532 [==============================] - 257s 2ms/step - loss: 0.3664 - f1: 0.8798
Epoch 1/2
133532/133532 [==============================] - 254s 2ms/step - loss: 0.3530 - f1: 0.8845
Epoch 2/2
133532/133532 [==============================] - 251s 2ms/step - loss: 0.3512 - f1: 0.8838
Epoch 1/2
133532/133532 [==============================] - 254s 2ms/step - loss: 0.3386 - f1: 0.8884
Epoch 2/2
133532/133532 [==============================] - 258s 2ms/step - loss: 0.3272 - f1: 0.8918
Epoch 1/2
133532/133532 [==============================] - 260s 2ms/step - loss: 0.3144 - f1: 0.8957
Epoch 2/2
133532/133532 [==============================] - 259s 2ms/step - loss: 0.3036 - f

KeyboardInterrupt: 

In [78]:
predict_label = model.predict(valid_data)
print(onehot2index(predict_label))
print(onehot2index(valid_label))
sklearn.metrics.f1_score(onehot2index(valid_label),onehot2index(predict_label),average='macro')

[4. 4. 4. ... 0. 0. 0.]
[4. 4. 4. ... 0. 0. 0.]


0.8201552493371331

In [29]:
def getValData():
    label = []
    data = []
    with open('valid_a_c.txt') as f:
        isFirst = True
        count = 1
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue

            if not line:
                break

            l = line.split('\t')

            label.append(int(l[5]))
            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
            
            
            
            data.append(new_result)
            if count%10000 == 0:
                print(count)
            count+=1
    
    #change lable to one hot
    label = np.array(label)
    ohe = OneHotEncoder()
    ohe.fit([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]])
    label = ohe.transform(label.reshape(label.size,1)).toarray()
    
    return (np.array(data),label)

(valid_data, valid_label) = getValData()

In [27]:
def getOutPutAndSave(model):
    data = []
    with open('test_a.txt') as f:
        isFirst = True
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue

            if not line:
                break

            l = line.split('\t')

            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
            
            data.append(new_result)
        
    data = np.array(data)
    
    result = model.predict(data)
    
    target = open('First.txt','w')
    
    for i in result:
        target.write(str(i.argmax()+1)+'\n')#输出
        
    target.close()

getOutPutAndSave(model)